In [1]:
%cd ../../

/home/PJWSTK/s14028/Engine


In [2]:
import keras

from common import *
from testing import test_model
from testing import best_cop_diff
from testing import mse
from testing import mae

from SDM.SDM import SDM

Using TensorFlow backend.


In [3]:
train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

In [4]:
sdm = SDM((480, 640, 3), split_into_parts=40)

images = sdm._prepare_images(image_test)
anwsers = sdm._prepare_anwsers(person_coo_test)

In [5]:
cop = np.linspace(0, 1, 11)[1:-1]

In [6]:
prefix = "SDM/SDM/weights/"

In [7]:
model_names = ["sdm_40_1e_2_470",
               "sdm_40_1e_2_500"]

model_names = [prefix + name for name in model_names]

In [8]:
models = [SDM((480, 640, 3), split_into_parts=40) for i in range(len(model_names))]

In [9]:
results = {}

for model_name, model in zip(model_names, models):
  model.def_model()
  model.model.fit(images[:1], anwsers[:1])
  
  model.model = keras.utils.multi_gpu_model(model.model, gpus=3, cpu_merge=False)
  model.model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
  
  model.model.load_weights(model_name)
  
  results[model_name] = test_model(model=model, images=image_test, counts=count_test, cut_of_points=cop)

Epoch 1/1
1/1 [==============================] - 3s 3s/step - loss: 0.6931 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 490ms/step - loss: 0.6931 - acc: 1.0000


In [10]:
for name, result in results.items():
  diff = best_cop_diff(result, count_test)
  
  print(f"Model {name} behaved:")
  
  print(f"For cut-of-point {diff[0]} had accuracy {diff[1]}:")
  print(diff[2])
  
  print(f"With MSE {mse(diff[2])}")
  print(f"With MAE {mae(diff[2])}", end="\n\n")

Model SDM/SDM/weights/sdm_40_1e_2_470 behaved:
For cut-of-point 0.1 had accuracy 70.7274955729962:
[ 12  16  14  19  16  22   3   0   7   6   5   5  -2   5   1   3   7   0
   6   7   5  10   3   3   4  10   8   5   9   6  16  -8   2   8  14  15
  19  18  22  10  18  20  29   8   7  13   8  20  17   5   6  11   4   2
   4   5  17   9   9   6  26   8   9   7   7   8   7  11   5   4   1  -1
   1   8   2   5   2   7   4   1   5   2   2   2   1   0  -2   6  13  -1
  12   2  14  14  18  -9  11  -5   5   2  26  10  14  19  24  15  15  25
   6  22  16  20  25  24  19  37  21  16  13   7  12  20  18  14   7   4
   8  12  14  15  23  21   4  21  29   9   6  17   4  12   4   6  10  11
   9  19   9   8  10   3   8   2   2  17   4  10  15  15  18   9   7   1
  17   5  -4   9   9   4  -2  10   0   4  12   7  11  16  15  -2  11   6
   8  13  10   7   4   6   7  10  10   2   1   9  12   1   8  19  16   9
  16  13   3   6   1   0   9   2   5  10   3   1  -2   9   2   8  -3   5
  13   8   8   5  11  10 